# Setup Variables

In [3]:
REPO_PATH = '/home/ah19/runtime-monitoring'
DATASET = 'MNIST'
PREFIX = 'Adam-256-30'
FILENAME_POSTFIX = f"{DATASET}_{PREFIX}"

NUM_NEURONS = PREFIX.split('-')[-1]
CLASSES_COLUMN = 'y'

FILENAME_POSTFIX

'MNIST_Adam-256-30'

# Libraries

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
import seaborn as sns
from pathlib import Path
import time

In [5]:
import sys
sys.path.append(REPO_PATH + '/utilities')
sys.path.append(f'{REPO_PATH}/{DATASET}/trainingModels')

In [6]:
from utils import *
from pcaFunctions import *
from scaleFunctions import *
from plotFunctions import *
from pathManager import fetchPaths

In [7]:
base = Path(REPO_PATH)
paths = fetchPaths(base, DATASET)

path = paths[DATASET.lower()]
path_dataset = paths['dataset']
path_lastHiddenLayer = paths['lastHiddenLayer']
path_lastHiddenLayer_raw = paths['lastHiddenLayer_raw'] / FILENAME_POSTFIX
path_lastHiddenLayer_pca = paths['lastHiddenLayer_pca'] / FILENAME_POSTFIX
path_savedModels = paths['savedModels']

# save model and logs to dedicated directory
path_lastHiddenLayer_pca_single = path_lastHiddenLayer_pca / 'Single'
path_lastHiddenLayer_pca_single.mkdir(exist_ok=True, parents=True)

path_lastHiddenLayer_pca_classes = path_lastHiddenLayer_pca / 'Classes'
path_lastHiddenLayer_pca_classes.mkdir(exist_ok=True, parents=True)

path, path_lastHiddenLayer_pca

(PosixPath('/home/ah19/runtime-monitoring/MNIST'),
 PosixPath('/home/ah19/runtime-monitoring/MNIST/lastHiddenLayer/pca/MNIST_Adam-256-30'))

# Import Data

In [8]:
df = pd.read_csv(path_lastHiddenLayer_raw / f'{FILENAME_POSTFIX}_train.csv')
f"Accuracy for train data: {df['true'].sum() / df.shape[0]}"

'Accuracy for train data: 0.9958666666666667'

In [9]:
df_test = pd.read_csv(path_lastHiddenLayer_raw / f'{FILENAME_POSTFIX}_test.csv')
f"Accuracy for test data: {df_test['true'].sum() / df_test.shape[0]}"

'Accuracy for test data: 0.9886'

### Export PCA - Single

In [11]:
scaler_single = fitStandardScalerSingle(df, NUM_NEURONS)
pca_single = fitPCASingle(df, scaler_single, NUM_NEURONS)

In [12]:
df_single = {
    'train': applyPCASingle(pca_single, df, scaler_single, NUM_NEURONS),
    'test': applyPCASingle(pca_single, df_test, scaler_single, NUM_NEURONS),
}

In [13]:
for stage, df in df_single.items():
    df.to_csv(path_lastHiddenLayer_pca_single / f'{FILENAME_POSTFIX}_{stage}.csv', index=False)

In [14]:
neurons_single = neuronsLoadingsSingle(pca_single, NUM_NEURONS, var_thld=0.9, loadings_thld=0.5)
save_json(path_lastHiddenLayer_pca_single / f'{FILENAME_POSTFIX}_neurons.json', neurons_single)

### Export PCA - Classes

In [15]:
scaler_classes = fitScalerClasses(df, CLASSES_COLUMN, NUM_NEURONS)
pca_classes = fitPCAClasses(df, CLASSES_COLUMN, scaler_classes, NUM_NEURONS)

In [17]:
df_classes = {
    'train': applyPCAClasses(pca_classes, df, CLASSES_COLUMN, scaler_classes),
    'test': applyPCAClasses(pca_classes, df_test, CLASSES_COLUMN, scaler_classes),
}

In [18]:
for stage, df in df_classes.items():
    df.to_csv(path_lastHiddenLayer_pca_classes / f'{FILENAME_POSTFIX}_{stage}.csv', index=False)

In [19]:
neurons_classes = neuronsLoadingsClasses(pca_classes, NUM_NEURONS, var_thld=0.9, loadings_thld=0.5)
save_json(path_lastHiddenLayer_pca_classes / f'{FILENAME_POSTFIX}_neurons.json', neurons_classes)